[![Binder](https://mybinder.org/badge_logo.svg)](https://nbviewer.org/github/vicente-gonzalez-ruiz/color-DCT/blob/main/color_DCT.ipynb)

[![Colab](https://badgen.net/badge/Launch/on%20Google%20Colab/blue?icon=notebook)](https://colab.research.google.com/github/vicente-gonzalez-ruiz/color_DCT/blob/main/color_DCT.ipynb)

# The Color-DCT

Removing redundancy in the color domain with the [DCT](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fftpack.dct.html).

## Deployment

In [ ]:
%%bash
if [ -d "$HOME/repos" ]; then
    echo "\"$HOME/repos\" exists"
else
    mkdir ~/repos
    echo Created $HOME/repos
fi

In [ ]:
%%bash
if [ -d "$HOME/repos/MRVC" ]; then
    cd $HOME/repos/MRVC
    echo "$HOME/repos/MRVC ... "
    git pull 
else
    cd $HOME/repos
    git clone https://github.com/Sistemas-Multimedia/MRVC.git
fi

In [ ]:
%%bash
if [ -d "$HOME/repos/image_IO" ]; then
    cd $HOME/repos/image_IO
    echo "$HOME/repos/image_IO ... "
    git pull 
else
    cd $HOME/repos
    git clone https://github.com/vicente-gonzalez-ruiz/image_IO.git
fi

In [ ]:
%%bash
if [ -d "$HOME/repos/scalar_quantization" ]; then
    cd $HOME/repos/scalar_quantization
    echo "$HOME/repos/scalar_quantization ... "
    git pull 
else
    cd $HOME/repos
    git clone https://github.com/vicente-gonzalez-ruiz/scalar_quantization.git
fi

In [ ]:
import os
try:
    import matplotlib
    %matplotlib inline
except:
    !pip install matplotlib
#import matplotlib.pyplot as plt
#import matplotlib.axes as ax
##plt.rcParams['text.usetex'] = True
##plt.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}'] #for \text command
#import pylab
#import math
try:
    import numpy as np
except:
    !pip install numpy
try:
    from scipy import signal
    from scipy.fftpack import dct, idct
except:
    !pip install scipy
try:
    import cv2
except:
    !pip install opencv-python
    !pip install opencv-python-headless # Binder compatibility
try:
    import colored
except:
    !pip install colored

!ln -sf ~/repos/image_IO/image_3.py .
!ln -sf ~/repos/image_IO/image_1.py .
!ln -sf ~/repos/scalar_quantization/quantization.py .
!ln -sf ~/repos/information_theory/information.py .
!ln -sf ~/repos/scalar_quantization/deadzone_quantization.py .

import color_DCT
import information
import image_3 as RGB_image
import image_1 as gray_image
import quantization
import deadzone_quantization as DZQ
#import colored

## Notebook parameters

In [ ]:
# Prefix of the RGB image to be compressed.

home = os.environ["HOME"]
fn = home + "/repos/MRVC/images/lena_color/"
#fn = home + "/MRVC/sequences/stockholm/"
image_dtype = np.uint8 # For 8 bpp/component images
#image_dtype = np.uint16 # For 16 bpp/component images

DCT_components = ['0', '1', '2']

## Color-DCT matrix computation
Inversely transform the posible deltas to get the 3x3 matrix coefficients.

In [ ]:
DCT_type = 3
norm = "ortho" # Orthonormal: orthogonal + unitary (unit gain in both directions of the transform)
#norm = None

In [ ]:
DCT0_delta = np.array([1, 0, 0])
idct(DCT0_delta, type=DCT_type, norm=norm)

In [ ]:
DCT0_delta = np.array([0, 1, 0])
idct(DCT0_delta, type=DCT_type, norm=norm)

In [ ]:
DCT0_delta = np.array([0, 0, 1])
idct(DCT0_delta, type=DCT_type, norm=norm)

## Read the image and show it

In [ ]:
RGB_img = RGB_image.read(fn).astype(image_dtype)
RGB_image.show(RGB_img, fn + "000.png")

In [ ]:
RGB_img.shape

In [ ]:
RGB_img.dtype

## (RGB -> color-DCT) transform of the image

In [ ]:
DCT_img = color_DCT.from_RGB(RGB_img)
print(DCT_img.dtype)

In [ ]:
gray_image.show(DCT_img[..., 0], fn + "000 (DCT0)")

In [ ]:
gray_image.show(DCT_img[..., 1], fn + "000 (DCT1)")

In [ ]:
gray_image.show(DCT_img[..., 2], fn + "000 (DCT2)")

## Energy of the DCT components

In [ ]:
DCT0_avg_energy = information.average_energy(DCT_img[..., 0])
DCT1_avg_energy = information.average_energy(DCT_img[..., 1])
DCT2_avg_energy = information.average_energy(DCT_img[..., 2])
print(f"Average energy of component DCT0 = {int(DCT0_avg_energy)}")
print(f"Average energy of component DCT1 = {int(DCT1_avg_energy)}")
print(f"Average energy of component DCT2 = {int(DCT2_avg_energy)}")
total_DCT_avg_energy = DCT0_avg_energy + DCT1_avg_energy + DCT2_avg_energy
print(f"Total average energy (computed by adding the energies of the DCT coefficients {int(DCT0_avg_energy)} + {int(DCT1_avg_energy)} + {int(DCT2_avg_energy)}) = {int(total_DCT_avg_energy)}")
print(f"Total RGB average energy (computed directly from the RGB image) = {int(information.average_energy(RGB_img)*3)}")

Therefore, the forward DCT is energy preserving (unitary).

In [ ]:
RGB_recons_img = color_DCT.to_RGB(DCT_img)
print(f"Total RGB average energy (computed from the reconstructed RGB image) = {int(information.average_energy(RGB_recons_img)*3)}")

And the same can be said about the backward transform.

## More insights about the orthogonality of the DCT
Orthogonal transforms preserve the energy and also, the distortion, that in last instance is a loss of energy.

In [ ]:
DCT_img = color_DCT.from_RGB(RGB_img)
Q = DZQ.Deadzone_Quantizer(Q_step=8)
quantized_DCT_img, k = Q.quan_dequan(DCT_img)
DCT_avg_energy = information.average_energy(quantized_DCT_img)
print(f"Average energy in the DCT domain = {int(DCT_avg_energy)}")

quantized_img, k = Q.quan_dequan(RGB_img)
RGB_avg_energy = information.average_energy(quantized_img)
print(f"Average energy in the RGB domain = {int(RGB_avg_energy)}")

Except for the error introduced by the arithmetic precision limit, the distortion can be measured in both domains, DCT and RGB. (Again) This is true because the DCT is orthogonal.

## (RGB <-> color-DCT) transform error
The DCT is only near-energy-preserving because the DCT coefficients are real (floating-point) numbers (only integer arithmetic is fully reversible).

In [ ]:
RGB_image.show_normalized(RGB_recons_img, fn + "000.png (DCT recons)")

In [ ]:
np.array_equal(RGB_img, RGB_recons_img)

In [ ]:
print(RGB_img.max(), RGB_img.min())

In [ ]:
print(RGB_recons_img.max(), RGB_recons_img.min())

In [ ]:
RGB_image.show_normalized(RGB_img - RGB_recons_img.astype(image_dtype), "Reconstruction error (rounding error) color-DCT")

The DCT transform is irreversible. In general, only integer arithmetic operations guarantees reversibility.

## Relative gains of the synthesis filters

The synthesis filters gains are important because the quantization steps of each color-DCT component should be adjusted in order to effectively provide the desired number of [bins](http://www.winlab.rutgers.edu/~crose/322_html/quantization.pdf) (different dequantized values) in each component.

In [ ]:
import numpy as np
from scipy.fftpack import dct, idct

DCT_type = 3
norm = "ortho" # Orthonormal: orthogonal + unitary (unit gain in both directions of the transform)
#norm = None

In [ ]:
def print_info(val):
    DCT0_delta = np.array([val, 0, 0])
    RGB_DCT0_delta = idct(DCT0_delta, type=DCT_type, norm=norm)
    RGB_energy_DCT0_delta = information.energy(RGB_DCT0_delta)
    
    DCT1_delta = np.array([0, val, 0])
    RGB_DCT1_delta = idct(DCT1_delta, type=DCT_type, norm=norm)
    RGB_energy_DCT1_delta = information.energy(RGB_DCT1_delta)
    
    DCT2_delta = np.array([0, 0, val])
    RGB_DCT2_delta = idct(DCT2_delta, type=DCT_type, norm=norm)
    RGB_energy_DCT2_delta = information.energy(RGB_DCT2_delta)
    
    zero = np.array([0, 0, 0])
    RGB_zero = idct(zero, type=DCT_type, norm=norm)
    RGB_energy_zero = information.energy(RGB_zero)
    
    print(f"{val}^2 = {val*val}")
    
    print(f"Energy of {DCT0_delta} in the RGB domain ({RGB_DCT0_delta}) = {RGB_energy_DCT0_delta}")
    print(f"Energy of {DCT1_delta} in the RGB domain ({RGB_DCT1_delta}) = {RGB_energy_DCT1_delta}")
    print(f"Energy of {DCT2_delta} in the RGB domain ({RGB_DCT2_delta}) = {RGB_energy_DCT2_delta}")
    print(f"Energy of {zero} in the RGB domain ({RGB_zero}) = {RGB_energy_zero}")
    
    max_ = max(RGB_energy_DCT0_delta, RGB_energy_DCT1_delta, RGB_energy_DCT2_delta)
    DCT0_relative_gain = RGB_energy_DCT0_delta / max_
    DCT1_relative_gain = RGB_energy_DCT1_delta / max_
    DCT2_relative_gain = RGB_energy_DCT2_delta / max_
    print(f"Relative gain of DCT0 component = {DCT0_relative_gain}")
    print(f"Relative gain of DCT1 component = {DCT1_relative_gain}")
    print(f"Relative gain of DCT2 component = {DCT2_relative_gain}")
    
print_info(255)
print()
print_info(1)
print()
print_info(0)

The gain of each color-DCT inverse filter is 1 (the transform is orthonormal). <!-- Therefore, under bit-rate restrictions, the optimal quantization pattern (without considering other aspects such as the compresibility of each component) is $\Delta_{\text{DCT0}} = \Delta_{\text{DCT1}} = \Delta_{\text{DCT2}}$. -->

## Amplitude shift in the DCT domain
This aspect can be interesting to encode the DCT coefficients.
<!--
To decide how to quantize, it is necessary to known how the amplitudes of the original image are *translated* to the transform domain. A good choice to find out this is to transform noise. In our case, lets use a random image with ([normal](https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html)) [Gaussian noise](https://en.wikipedia.org/wiki/Gaussian_noise) with mean 0, color-DCT it, and check where the transformed noise has its mean in each component. Notice that, by definition, the noise cannot be decorrelated by transforms, and therefore, the noise is simply *transfered* from the RGB domain to the transform domain. Thus, depending on where the transformed noise has the mean, we can decide if the signal must be shifted before or after the transform. Notice that the input signal to a dead-zone quantizer must have 0 mean.-->

In [ ]:
# loc = mean, scale=standard deviation, size=number of samples
RGB_noise = np.random.normal(loc=0, scale=10, size=512*512*3).reshape(512, 512, 3)

In [ ]:
RGB_image.show(RGB_noise, "Gaussian RGB noise")

In [ ]:
DCT_noise = color_DCT.from_RGB(RGB_noise)

In [ ]:
gray_image.show_normalized(DCT_noise[..., 0], "Gaussian DCT0 noise")

In [ ]:
gray_image.show_normalized(DCT_noise[..., 1], "Gaussian DCT1 noise")

In [ ]:
gray_image.show_normalized(DCT_noise[..., 2], "Gaussian DCT2 noise")

The color-DCT does not modify the mean of the image when it has a 0 mean.

## Conclusion

* The Color-DCT concentrates most of the energy in the DCT$_0$ component, that represents the luminance of the image.
* The DCT domain is irreversible and needs floating-point arithmetic.

<!-- ## Entropy gain
Let's check if the energy compactation reduces the amount of information (that can be quantified in terms of the entropy). Notice that the DCT coefficients are real numbers, and for this reason, they need to be quantized before the entropy is computed. In this case, we will truncate the -->